# import

In [1]:
import pandas as pd
target_table = pd.read_csv(r'xxx.csv')
target_table

FileNotFoundError: [Errno 2] No such file or directory: 'xxx.csv'

# function

    やりたいこととしては、UU割合10%でセグメントを10分割するのだが、
    0の存在がノイズになるので、0と0以外の実数を分けて、0のセグメントのUUと継続率と、0以外の実数のセグメントのUUと継続率を算出する

# 値ベースでセグメントをN等分に分割

In [229]:
# DataFrame から特定の列を選択し、`column_one` 列で降順にソート
data = target_table[['user_id', 'login_flg',　'column_one']].sort_values(by ='column_one' , ascending = False)

# 0以外の処理
# uu, uu_rate, churn_rate
data_other_real_number = data[data.iloc[:,2] != 0]
data_other_real_number = data_other_real_number[['user_id', 'login_flg','column_one']].sort_values(by ='column_one' , ascending = False)

data_other_real_number.loc[:, 'threshold'] = pd.qcut(data_other_real_number.iloc[:, 2], q=10, duplicates='drop')
data_other_real_number.loc[:, 'label'] = pd.qcut(data_other_real_number.iloc[:, 2], q=10, labels=False, duplicates='drop')
data_other_real_number_threshold = data_other_real_number[['label', 'threshold']].drop_duplicates(subset = 'threshold')

uu = pd.DataFrame(data_other_real_number.groupby(['label'], as_index= False, dropna = False)['user_id'].count()).rename(columns={'user_id': 'uu'})
uu_churn = pd.DataFrame(data_other_real_number[data_other_real_number['login_flg'] == 0].groupby(['label'], as_index= False, dropna = False)['user_id'].count()).rename(columns={'user_id': 'uu_churn'})
df_churn = pd.merge(uu, uu_churn, how = 'left', on ='label')
df_churn['churn_rate'] = df_churn['uu_churn'] / df_churn['uu']

df_churn = pd.merge(df_churn, data_other_real_number_threshold, how = 'left', on ='label').sort_values(by = 'label', ascending = False)

# 0の処理
# uu, uu_rate, churn_rate
data_only_zero = data[data.iloc[:, 2] == 0]
uu = data_only_zero['user_id'].count()
uu_churn = data_only_zero[data_only_zero['login_flg'] == 0]['user_id'].count()
churn_rate = uu_churn / uu
df_churn_only_zero = pd.DataFrame({'label': [-999], 'uu': [uu], 'uu_churn': [uu_churn], 'churn_rate': [churn_rate], 'threshold': [0]})

df_result = pd.concat([df_churn, df_churn_only_zero], axis = 0)
df_result


,label,uu,uu_churn,churn_rate,threshold
9,9,333,29,0.087087,"(11780.0, 63400.0]"
8,8,305,21,0.068852,"(9600.0, 11780.0]"
7,7,355,24,0.067606,"(8400.0, 9600.0]"
6,6,308,25,0.081169,"(7400.0, 8400.0]"
5,5,345,25,0.072464,"(6000.0, 7400.0]"
4,4,330,38,0.115152,"(4200.0, 6000.0]"
3,3,335,30,0.089552,"(2600.0, 4200.0]"
2,2,331,23,0.069486,"(1200.0, 2600.0]"
1,1,355,26,0.073239,"(380.0, 1200.0]"
0,0,334,28,0.083832,"(99.999, 380.0]"


# インデックスベースでセグメントを10等分に分割

In [5]:
# # 以下のセルでの処理のためのテスト
# # データの長さを取得
# data_length = len(target_table['columns_one'])

# # %の地点を計算
# _10_percent_index = int(data_length * 0.1)
# _20_percent_index = int(data_length * 0.2)
# _30_percent_index = int(data_length * 0.3)
# _40__percent_index = int(data_length * 0.4)
# _50_percent_index = int(data_length * 0.5)
# _60_percent_index = int(data_length * 0.6)
# _70_percent_index = int(data_length * 0.7)
# _80_percent_index = int(data_length * 0.8)
# _90_percent_index = int(data_length * 0.9)
# _100_percent_index = int(data_length * 1)

# # %の地点の行を抽出
# _10_percent_row = senryokuzokyo.iloc[0:_10_percent_index,:]
# _20_percent_row = senryokuzokyo.iloc[_10_percent_index:_20_percent_index,:]
# _30_percent_row = senryokuzokyo.iloc[_20_percent_index:_30_percent_index,:]
# _40_percent_row = senryokuzokyo.iloc[_30_percent_index:_40__percent_index,:]
# _50_percent_row = senryokuzokyo.iloc[_40__percent_index:_50_percent_index,:]
# _60_percent_row = senryokuzokyo.iloc[_50_percent_index:_60_percent_index,:]
# _70_percent_row = senryokuzokyo.iloc[_60_percent_index:_70_percent_index,:]
# _80_percent_row = senryokuzokyo.iloc[_70_percent_index:_80_percent_index,:]
# _90_percent_row = senryokuzokyo.iloc[_80_percent_index:_90_percent_index,:]
# _100_percent_row = senryokuzokyo.iloc[_90_percent_index:_100_percent_index,:]

# uu = _10_percent_row.user_id.count()
# uu_churn = _10_percent_row[_10_percent_row['login_flg'] == 0]['user_id'].count()
# churn_late = uu_churn / uu
# df_result = pd.DataFrame({'uu': [uu], 'uu_churn': [uu_churn], 'churn_rate': [churn_late]})
# df_result

In [230]:
# データの長さを取得
data_length = len(data_other_real_number)

# 各パーセンテージ地点の行と閾値を抽出
percent_rows = []
thresholds = []
for i in range(1, 11):
    start_index = int(data_length * ((i - 1) / 10))
    end_index = int(data_length * (i / 10))
    percent_row = data_other_real_number.iloc[start_index:end_index, :]
    threshold = data_other_real_number.iloc[end_index - 1, 2]  # ここで閾値を取得
    percent_rows.append(percent_row)
    thresholds.append(threshold)

# 各パーセンテージ地点ごとに uu、uu_churn、churn_rate を計算して結果をデータフレームに追加
dfs_result = []
for i, (percent_row, threshold) in enumerate(zip(percent_rows, thresholds), start=1):
    uu = percent_row['user_id'].count()
    uu_churn = percent_row[percent_row['login_flg'] == 0]['user_id'].count()
    churn_rate = uu_churn / uu
    df_result = pd.DataFrame({'uu': [uu], 'uu_churn': [uu_churn], 'churn_rate': [churn_rate], 'threshold': [threshold]})
    dfs_result.append(df_result)

# 結果のデータフレームを縦に結合
df_result = pd.concat(dfs_result, ignore_index=True)

# 0の処理
data_only_zero = data[data.iloc[:, 2] == 0]
uu = data_only_zero['user_id'].count()
uu_churn = data_only_zero[data_only_zero['login_flg'] == 0]['user_id'].count()
churn_rate = uu_churn / uu
df_churn_only_zero = pd.DataFrame({'uu': [uu], 'uu_churn': [uu_churn], 'churn_rate': [churn_rate], 'threshold': [0]})

df_final_result = pd.concat([df_result, df_churn_only_zero], axis = 0)
df_final_result

,uu,uu_churn,churn_rate,threshold
0,333,29,0.087087,11790
1,333,22,0.066066,9600
2,333,24,0.072072,8400
3,333,27,0.081081,7400
4,333,26,0.078078,6000
5,333,34,0.102102,4200
6,333,30,0.090090,2600
7,333,24,0.072072,1200
8,333,25,0.075075,390
9,334,28,0.083832,100


## セグメント(ルールベース)

In [ ]:
# 50区切り
def segment_fifty(value):
    if value == 0:
        return '0'
    elif value < 50:
        return '1~49'
    elif value < 100:
        return '50~99'
    elif value < 150:
        return '100~149'
    elif value < 200:
        return '150~199'
    elif value < 250:
        return '200~249'
    elif value < 300:
        return '250~299'
    elif value < 350:
        return '300~349'
    elif value < 400:
        return '350~399'
    elif value < 450:
        return '400~449'
    elif value < 500:
        return '450~499'
    elif value >= 500:
        return '500~'
    else:
        return '0'

In [ ]:
# 100区切り
def segment_one_hundred(value):
    if value == 0:
        return '0'
    elif value < 100:
        return '1~99'
    elif value < 200:
        return '100~199'
    elif value < 300:
        return '200~299'
    elif value < 400:
        return '300~399'
    elif value < 500:
        return '400~499'
    elif value < 600:
        return '500~599'
    elif value < 700:
        return '600~699'
    elif value < 800:
        return '700~799'
    elif value < 900:
        return '800~899'
    elif value < 1000:
        return '900~999'
    elif value >= 1000:
        return '1,000~'
    else:
        return '0'

In [ ]:
# 500区切り
def segment_five_hundred(value):
    if value == 0:
        return '0'
    elif value < 500:
        return '1~499'
    elif value < 1000:
        return '500~999'
    elif value < 1500:
        return '1,000~1,499'
    elif value < 2000:
        return '1,500~1,999'
    elif value < 2500:
        return '2,000~2,499'
    elif value < 3000:
        return '2,500~2,999'
    elif value < 3500:
        return '3,000~3,499'
    elif value < 4000:
        return '3,500~3,999'
    elif value < 4500:
        return '4,000~4,499'
    elif value < 5000:
        return '4,500~4,999'
    elif value >= 5000:
        return '5,000~'
    else:
        return '0'

# def uu_rate_segment(value):
#     ranges = [(0, '0'), (600, '0~'), (1200, '600~'), (1700, '1200~'), (2300, '1700~'), (2900, '2300~'), 
#               (3500, '2900~'), (4000, '3500~'), (4600, '4000~'), (5200, '4600~'), (5800, '5200~')]
    
#     for limit, label in ranges:
#         if value < limit:
#             return label
#     return '5800~'  # 最後の範囲外の場合

In [ ]:
# 1000区切り
def segment_one_thounsand(value):
    if value == 0:
        return '0'
    elif value < 1000:
        return '1~999'
    elif value < 2000:
        return '1,000~1,999'
    elif value < 3000:
        return '2,000~2,999'
    elif value < 4000:
        return '3,000~3,999'
    elif value < 5000:
        return '4,000~4,999'
    elif value < 6000:
        return '5,000~5,999'
    elif value < 7000:
        return '6,000~6,999'
    elif value < 8000:
        return '7,000~7,999'
    elif value < 9000:
        return '8,000~8,999'
    elif value < 10000:
        return '9,000~9,999'
    elif value >= 10000:
        return '10,000~'
    else:
        return '0'

In [ ]:
# 1500区切り
def segment_fifteen_thounsand(value):
    if value == 0:
        return '0'
    elif value < 1500:
        return '1~1,499'
    elif value < 3000:
        return '1,500~2,999'
    elif value < 4500:
        return '3,000~4,499'
    elif value < 6000:
        return '4,500~5,999'
    elif value < 7500:
        return '6,000~7,499'
    elif value < 9000:
        return '7,500~8,999'
    elif value < 10500:
        return '9,000~10,499'
    elif value < 12000:
        return '10,500~11,999'
    elif value < 13500:
        return '12,000~13,499'
    elif value < 15000:
        return '13,500~14,999'
    elif value >= 15000:
        return '15,000~'
    else:
        return '0'

In [ ]:
# 2000区切り
def segment_twenty_thounsand(value):
    if value == 0:
        return '0'
    elif value < 2000:
        return '1~1,999'
    elif value < 4000:
        return '2,000~3,999'
    elif value < 6000:
        return '4,000~5,999'
    elif value < 8000:
        return '6,000~7,999'
    elif value < 10000:
        return '8,000~9,999'
    elif value < 12000:
        return '10,000~11,999'
    elif value < 14000:
        return '12,000~13,999'
    elif value < 16000:
        return '14,000~15,999'
    elif value < 18000:
        return '16,000~17,999'
    elif value < 20000:
        return '18,000~19,999'
    elif value >= 20000:
        return '20,000~'
    else:
        return '0'

In [ ]:
# 機体数セグメント(3機体)
def machine_segment(value):
    if value == 0:
        return '0'
    elif value < 4:
        return '1~3'
    elif value < 7:
        return '4~6'
    elif value < 10:
        return '7~9'
    elif value < 13:
        return '10~12'
    elif value < 16:
        return '13~15'
    elif value >= 16:
        return '16~'
    else:
        return '0'

In [ ]:
# インデックスベースでビンを10等分に区切った結果のセグメント
def segment_1(value):
    if value == 0:
        return '0'
    elif value < 100:
        return '1~99'
    elif value < 200:
        return '100~199'
    elif value < 300:
        return '200~299'
    elif value == 300:
        return '300'
    elif value < 400:
        return '301~399'
    elif value < 500:
        return '400~499'
    elif value < 600:
        return '500~599'
    elif value < 700:
        return '600~699'
    elif value < 1000:
        return '700~999'
    elif value >= 1000:
        return '1,000~'
    else:
        return '0'

# インデックスベースでビンを10等分に区切った結果のセグメント
def segment_2(value):
    if value == 0:
        return '0'
    elif value < 100:
        return '1~99'
    elif value < 190:
        return '100~189'
    elif value < 200:
        return '190~199'
    elif value < 300:
        return '200~299'    
    elif value == 300:
        return '300'
    elif value < 400:
        return '301~399'
    elif value < 500:
        return '400~499'
    elif value < 600:
        return '500~599'
    elif value < 800:
        return '600~799'
    elif value >= 800:
        return '800~'
    else:
        return '0'

In [ ]:
# セグメントを適用するカラムを指定
column = ['column_one', 'column_two', 'column_three', 'column_four', 'column_five']

# セグメントを適用して新しい列を追加
for col in column:
    new_col = f'segment_uu_{col}'  # 新しい列の名前を定義
    target_table[new_col] = target_table[col].apply(segment_fifty)

target_table

NameError: name 'target_table' is not defined

In [ ]:
# ルールベースでのuu, uu_rate, churn_rateの算出
# login_flg: 1/0(1:継続, 0:非継続)
# login_flg: sum -> 継続した人
# login_flg': count ->  ログインした人
# 継続した人 / ログインした人 -> 継続率

segment_one_u = target_table.groupby('segment_one_uu', as_index=False, dropna=False).agg({'login_flg': ['sum', 'count']})
segment_one_u.columns = segment_one_u.columns.map('_'.join)
segment_one_u['diff'] = segment_one_u['login_flg_count'] - segment_one_u['login_flg_sum']
segment_one_u['churn_rate'] = segment_one_u['diff'] / segment_one_u['login_flg_count']
segment_one_u

NameError: name 'target_table' is not defined